In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
df = pd.read_csv('france_data.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,01-12-2010 08:45,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,01-12-2010 08:45,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,01-12-2010 08:45,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,01-12-2010 08:45,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,01-12-2010 08:45,0.65,12583.0,France


In [4]:
df["Bill_Value"]=df["UnitPrice"]*df["Quantity"]
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Bill_Value
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,01-12-2010 08:45,3.75,12583.0,France,90.0
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,01-12-2010 08:45,3.75,12583.0,France,90.0
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,01-12-2010 08:45,3.75,12583.0,France,45.0
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,01-12-2010 08:45,0.85,12583.0,France,10.2
4,536370,21883,STARS GIFT TAPE,24,01-12-2010 08:45,0.65,12583.0,France,15.6


In [10]:
from datetime import datetime
import datetime
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"])

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Bill_Value
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-01-12 08:45:00,3.75,12583.0,France,90.0
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,2010-01-12 08:45:00,3.75,12583.0,France,90.0
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-01-12 08:45:00,3.75,12583.0,France,45.0
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,2010-01-12 08:45:00,0.85,12583.0,France,10.2
4,536370,21883,STARS GIFT TAPE,24,2010-01-12 08:45:00,0.65,12583.0,France,15.6


In [11]:
df.dtypes

InvoiceNo               int64
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
Bill_Value            float64
dtype: object

In [12]:
df["Bill_Value"]=pd.to_numeric(df["Bill_Value"])
df["CustomerID"]=pd.to_numeric(df["CustomerID"])

In [13]:
df["CustomerID"].count()

8342

# Recency

In [14]:
#group by customers and check last date of purshace
recency_df = df.groupby(by='CustomerID', as_index=False)['InvoiceDate'].max()
recency_df.columns = ['Customer_ID','LastPurshaceDate']
recency_df.head()

,Customer_ID,LastPurshaceDate
0,12413.0,2011-06-01 16:12:00
1,12437.0,2011-12-01 14:13:00
2,12441.0,2010-08-12 10:45:00
3,12488.0,2011-11-30 08:50:00
4,12489.0,2011-07-01 13:13:00


In [16]:
from datetime import datetime as dt
now=(df["InvoiceDate"].max())
now

Timestamp('2011-12-10 14:46:00')

In [17]:
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (now - x).days)

In [18]:
recency_df.head()

,Customer_ID,LastPurshaceDate,Recency
0,12413.0,2011-06-01 16:12:00,191
1,12437.0,2011-12-01 14:13:00,9
2,12441.0,2010-08-12 10:45:00,485
3,12488.0,2011-11-30 08:50:00,10
4,12489.0,2011-07-01 13:13:00,162


In [19]:
#now let us delete LastPurchaseDate as we do not need it any more we will be dealing with only the recency value
recency_df.drop(["LastPurshaceDate"], axis=1, inplace=True)
recency_df.head()

,Customer_ID,Recency
0,12413.0,191
1,12437.0,9
2,12441.0,485
3,12488.0,10
4,12489.0,162


# Frequency

In [26]:
frequency=df.groupby(by=['CustomerID'], as_index=False)['Bill_Value'].count()
frequency.columns=["Customer_ID", "Frequency"]
frequency.head()

,Customer_ID,Frequency
0,12413.0,38
1,12437.0,200
2,12441.0,11
3,12488.0,55
4,12489.0,15


# Monetary

In [28]:
avg=df.groupby("CustomerID", as_index=False)['Bill_Value'].mean()
avg.columns=["Customer_ID", "Monetary"]
avg.head()

,Customer_ID,Monetary
0,12413.0,19.950000
1,12437.0,24.757050
2,12441.0,15.777273
3,12488.0,23.612000
4,12489.0,22.328667


In [35]:
rfm=recency_df.merge(frequency, on="Customer_ID")
rfm2=rfm.merge(avg, on="Customer_ID")
rfm2.head()

,Customer_ID,Recency,Frequency,Monetary
0,12413.0,191,38,19.950000
1,12437.0,9,200,24.757050
2,12441.0,485,11,15.777273
3,12488.0,10,55,23.612000
4,12489.0,162,15,22.328667


# Customer segments with RFM Model
The simplest way to create customers segments from RFM Model is to use Quartiles. We assign a score from 1 to 4 to Recency, Frequency and Monetary. Four is the best/highest value, and one is the lowest/worst value. A final RFM score is calculated simply by combining individual RFM score numbers.

Note: Quintiles (score from 1-5) offer better granularity, in case the business needs that but it will be more challenging to create segments since we will have 555 possible combinations. So, we will use quartiles.

In [39]:
rfm_df =rfm2
#rfm_df.set_index('Customer_ID',inplace=True)
rfm_df.head()

,Recency,Frequency,Monetary
Customer_ID,,,
12413.0,191,38,19.950000
12437.0,9,200,24.757050
12441.0,485,11,15.777273
12488.0,10,55,23.612000
12489.0,162,15,22.328667


In [47]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary
0.25,15.5,23.5,16.564840
0.50,40.0,47.0,19.460566
0.75,143.5,125.0,23.045019


In [48]:
quantiles.to_dict()

{'Frequency': {0.25: 23.5, 0.5: 47.0, 0.75: 125.0},
 'Monetary': {0.25: 16.564839743589744,
  0.5: 19.460566037735845,
  0.75: 23.045018939393938},
 'Recency': {0.25: 15.5, 0.5: 40.0, 0.75: 143.5}}

# Creation of RFM Segments
We will create two segmentation classes since, high recency is bad, while high frequency and monetary value is good.

In [49]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [50]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [51]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
Customer_ID,,,,,,
12413.0,191,38,19.950000,1,2,3
12437.0,9,200,24.757050,4,4,4
12441.0,485,11,15.777273,1,1,1
12488.0,10,55,23.612000,4,3,4
12489.0,162,15,22.328667,1,1,3


Now that we have the score of each customer, we can represent our customer segmentation. First, we need to combine the scores (R_Quartile, F_Quartile,M_Quartile) together.

In [52]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
Customer_ID,,,,,,,
12413.0,191,38,19.950000,1,2,3,123
12437.0,9,200,24.757050,4,4,4,444
12441.0,485,11,15.777273,1,1,1,111
12488.0,10,55,23.612000,4,3,4,434
12489.0,162,15,22.328667,1,1,3,113


Best Recency score = 4: most recently purchase. Best Frequency score = 4: most quantity purchase. Best Monetary score = 4: spent the most.

Let's see who are our Champions (best customers).

In [53]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False).head(10)

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
Customer_ID,,,,,,,
12583.0,0,247,29.479271,4,4,4,444
12727.0,11,139,28.638129,4,4,4,444
12437.0,9,200,24.757050,4,4,4,444
12683.0,5,344,24.641192,4,4,4,444
12682.0,1,525,23.406133,4,4,4,444


# How many customers do we have in each segment?

In [54]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Best Customers:  5
Loyal Customers:  22
Big Spenders:  22
Almost Lost:  1
Lost Customers:  0
Lost Cheap Customers:  4


# Market Basket Analysis

In [55]:
#creating a user*product matrix
basket = pd.pivot_table(df,index='InvoiceNo', columns='Description',values='Quantity', fill_value=0)
basket.shape

(392, 1563)

In [56]:
#function to replace natural number with 1
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1) #dropping off the records which have been posted


In [57]:
#applying apriori algorithm to generate rules which have support more than 0.05
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.071429,[4 TRADITIONAL SPINNING TOPS]
1,0.096939,[ALARM CLOCK BAKELIKE GREEN]
2,0.102041,[ALARM CLOCK BAKELIKE PINK]
3,0.094388,[ALARM CLOCK BAKELIKE RED]
4,0.068878,[ASSORTED COLOUR MINI CASES]


In [58]:
#calculating confidence and lift for the genrated rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061


In [59]:
#selecting only the rules which have lift >6 and confidence >0.8
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
8,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.068878,0.071429,0.063776,0.925926,12.962963,0.058856,12.535714
9,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.071429,0.068878,0.063776,0.892857,12.962963,0.058856,8.690476
38,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 6 SKULL PAPER CUPS),0.056122,0.063776,0.051020,0.909091,14.254545,0.047441,10.298469
39,(PACK OF 6 SKULL PAPER CUPS),(PACK OF 6 SKULL PAPER PLATES),0.063776,0.056122,0.051020,0.800000,14.254545,0.047441,4.719388
64,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
66,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
67,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
68,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...",(ALARM CLOCK BAKELIKE GREEN),0.073980,0.096939,0.063776,0.862069,8.892922,0.056604,6.547194
